In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Time-series data:

In [ ]:
filename = "data.csv"
data = np.genfromtxt(filename, delimiter=",", skip_header=1)

# [t, y1, y2, y3, y4, y5] = [data[:, i] for i in range(6)]

t = data[:, 0]
ydata = [data[:, i] for i in range(1, 6)]

In [ ]:
for i, y in enumerate(ydata):
    plt.plot(t, y, label=f"data:{i+1}")
plt.legend()
plt.show()

### Autocorrelation

In [ ]:
def acf(data):
    tdata = data - np.mean(data)
    tmp = np.correlate(tdata, tdata, "full") / np.var(tdata) / tdata.size
    # This is reversed and repeated. Take first half, and reverse it:
    return np.flip(tmp[: int(len(tmp) / 2) + 1])

In [ ]:
for i, y in enumerate(ydata):
    a = acf(y)
    plt.plot(t, a, label=f"data:{i+1}")
plt.xlim(0, 100)
plt.title("Autocorrelation")
plt.legend()
plt.show()

## Fourier:

In [ ]:
filename2 = "fdata.csv"
data2 = np.genfromtxt(filename2, delimiter=",", skip_header=1)

[t2, f1, f2, f3] = [data2[:, i] for i in range(4)]

In [ ]:
plt.plot(t2, f2, label="f2")
plt.plot(t2, f1, label="f1")
plt.plot(t2, f3, label="f3")
plt.legend()
plt.show()

In [ ]:
from scipy.fft import fft, fftshift, fftfreq

dt = t2[1] - t2[0]

ft1 = fftshift(fft(f1))
ft2 = fftshift(fft(f2))
ft3 = fftshift(fft(f3))

freq = fftshift(fftfreq(f1.size, d=dt))

amp1 = 2.0 * np.abs(ft1) / len(t)
amp2 = 2.0 * np.abs(ft2) / len(t)
amp3 = 2.0 * np.abs(ft3) / len(t)

plt.plot(freq, amp1, "-", label="amplitude 1")
plt.plot(freq, amp2, "--", label="amplitude 2")
plt.plot(freq, amp3, "-.", label="amplitude 3")
plt.xlim(0)
plt.ylabel("Amplitude")
plt.xlabel("f (Hz)")
plt.legend()
plt.show()

In [ ]:
a1 = acf(f1)
a2 = acf(f2)
a3 = acf(f3)


plt.plot(t2, a1)
plt.plot(t2, a2)
plt.plot(t2, a3)
plt.xlim(0, 0.2)
plt.show()

## CO2 data

In [ ]:
co2_fname = "co2_mm_gl.csv"
co2_data = np.genfromtxt(co2_fname, delimiter=",", skip_header=38, names=True)
x = co2_data["decimal"]
y = co2_data["average"]

In [ ]:
plt.plot(co2_data["decimal"], co2_data["average"])
plt.plot(co2_data["decimal"], co2_data["trend"])
plt.show()

In [ ]:
coefs = np.polyfit(x, y, 1)
yfit = np.poly1d(coefs)

y1 = y - yfit(x)

plt.plot(x, y1, ".-")
plt.xlabel("year")
plt.ylabel("CO$_2$ (ppm), with trend subtracted")
plt.show()

In [ ]:
coefs = np.polyfit(x, y, 2)
yfit = np.poly1d(coefs)

y2 = y - yfit(x)

plt.plot(x, y2, ".-")
plt.xlabel("year")
plt.ylabel("CO$_2$ (ppm), with trend subtracted")
plt.show()

In [ ]:
from scipy import signal

y_smooth_12 = signal.savgol_filter(y1, window_length=6, polyorder=2)
y_smooth_24 = signal.savgol_filter(y1, window_length=24, polyorder=1)
plt.plot(x, y_smooth_12, label="2nd order, 6 month window")
plt.plot(x, y_smooth_24, label="1st order, 2 year window")
plt.xlabel("Year")
plt.ylabel("Deviation from 1st-order poly trend")
plt.show()

In [ ]:
y_smooth_12 = signal.savgol_filter(y2, window_length=6, polyorder=2)
y_smooth_24 = signal.savgol_filter(y2, window_length=24, polyorder=1)
plt.plot(x, y_smooth_12, label="2nd order, 6 month window")
plt.plot(x, y_smooth_24, label="1st order, 2 year window")
plt.xlabel("Year")
plt.ylabel("Deviation from 2nd-order poly trend")
plt.show()

In [ ]:
import seaborn as sns

nrows = 1
ncols = round(y.size / nrows) + 1

yt = np.resize(y, ncols * nrows)
z = yt.reshape((nrows, ncols))

ax = sns.heatmap(z)

plt.title("CO$_2$ (ppm)")
plt.ylabel("Cycles")
plt.xlabel("Months since Jan 1979")
plt.show()

In [ ]:
yt = np.resize(y1, ncols * nrows)
z = yt.reshape((nrows, ncols))

ax = sns.heatmap(z)

plt.title("CO$_2$ (ppm) - deviation from linear trend")
plt.ylabel("Cycles")
plt.xlabel("Months since Jan 1979")
plt.show()

In [ ]:
import datetime

# Month labels:
months = [datetime.date(1900, i + 1, 1).strftime("%B") for i in range(12)]

ncols = 12
nrows = round(y.size / ncols)

yt = np.resize(y1, ncols * nrows)
z = yt.reshape((nrows, ncols))

ax = sns.heatmap(z)
ax.invert_yaxis()
ax.set_xticklabels(months, rotation=50)

# Set y-tick locations: 0 to nrows, every 2nd:
ax.set_yticks(range(0, nrows, 2))
# Year labels (every 2nd):
tyears = [f"{1979+2*i}" for i in range(int(nrows / 2 + 1))]
# Set years as labels
ax.set_yticklabels(tyears)

plt.title("CO$_2$ (ppm) - deviation from linear trend")
plt.show()

In [ ]:
yt = np.resize(y2, ncols * nrows)
z = yt.reshape((nrows, ncols))

ax = sns.heatmap(z)
ax.invert_yaxis()
ax.set_xticklabels(months, rotation=50)

# Set y-tick locations: 0 to nrows, every 2nd:
ax.set_yticks(range(0, nrows, 2))
# Year labels (every 2nd):
tyears = [f"{1979+2*i}" for i in range(int(nrows / 2 + 1))]
# Set years as labels
ax.set_yticklabels(tyears)

plt.title("CO$_2$ (ppm) - deviation from quadratic trend")
plt.show()

## Cycle plot:

* First, see how we can filter the data to get a plot for each month:

In [ ]:
jan_data = co2_data[co2_data["month"] == 1]
feb_data = co2_data[co2_data["month"] == 2]

plt.plot(jan_data["year"], jan_data["average"], label="Jan")
plt.plot(feb_data["year"], feb_data["average"], label="Feb")
plt.show()

### Use subplots to begin our cycle plot

In [ ]:
fig, axs = plt.subplots(ncols=12, sharey=True)

for i, ax in enumerate(axs):
    month_data = co2_data[co2_data["month"] == i + 1]
    ax.plot(month_data["year"], month_data["average"])

### ... lets make this nicer:

In [ ]:
def i_to_month(i):
    # presumably there's a nicer way to do this..
    return datetime.date(1900, i, 1).strftime("%b")

In [ ]:
fig, axs = plt.subplots(ncols=12, sharey=True)
plt.subplots_adjust(wspace=0.0)

for i, ax in enumerate(axs):
    month_data = co2_data[co2_data["month"] == i + 1]
    ax.plot(month_data["year"], month_data["average"])
    # add x,o to first/last year; only add label (for legend) once:
    label1 = "1979" if i == 0 else None
    label2 = "2023" if i == 0 else None
    ax.plot(month_data["year"][0], month_data["average"][0], "bx", label=label1)
    ax.plot(month_data["year"][-1], month_data["average"][-1], "bo", label=label2)
    ax.set_xlim(1979 - 10, 2023 + 10)  # leave some space on either side
    ax.set_xlabel(i_to_month(i + 1))
    ax.set_xticks([])
    ax.set_frame_on(False)
    ax.grid(True)
    ax.tick_params(left=False, bottom=False)
    if i == 0:
        ax.set_ylabel("CO$_2$ (ppm)")
plt.suptitle("CO$_2$")
fig.legend()
plt.show()

In [ ]:
co2_data_copy = co2_data.copy()
co2_data_copy["average"] = y1

fig, axs = plt.subplots(ncols=12, sharey=True)
plt.subplots_adjust(wspace=0.0)

for i, ax in enumerate(axs):
    month_data = co2_data_copy[co2_data["month"] == i + 1]
    ax.plot(month_data["year"], month_data["average"])

    # add x,o to first/last year; only add label (for legend) once:
    label1 = "1979" if i == 0 else None
    label2 = "2023" if i == 0 else None
    label3 = str(int(month_data["year"][21])) if i == 0 else None
    ax.plot(month_data["year"][0], month_data["average"][0], "bx", label=label1)
    ax.plot(month_data["year"][21], month_data["average"][21], "b+", label=label3)
    ax.plot(month_data["year"][-1], month_data["average"][-1], "bo", label=label2)

    ax.set_xlim(1979 - 10, 2023 + 10)  # leave some space on either side
    ax.set_xlabel(i_to_month(i + 1))
    ax.set_xticks([])
    ax.set_frame_on(False)
    ax.grid(True)
    ax.tick_params(left=False, bottom=False)
    if i == 0:
        ax.set_ylabel("CO$_2$ concentration (ppm)")
plt.suptitle("CO$_2$: deviation from linear trend")
fig.legend()
plt.show()

In [ ]:
co2_data_copy = co2_data.copy()
co2_data_copy["average"] = y2

fig, axs = plt.subplots(ncols=12, sharey=True)
plt.subplots_adjust(wspace=0.0)

for i, ax in enumerate(axs):
    month_data = co2_data_copy[co2_data["month"] == i + 1]
    ax.plot(month_data["year"], month_data["average"])

    # add x,o to first/last year; only add label (for legend) once:
    label1 = "1979" if i == 0 else None
    label2 = "2023" if i == 0 else None
    label3 = str(int(month_data["year"][21])) if i == 0 else None
    ax.plot(month_data["year"][0], month_data["average"][0], "bx", label=label1)
    ax.plot(month_data["year"][21], month_data["average"][21], "b+", label=label3)
    ax.plot(month_data["year"][-1], month_data["average"][-1], "bo", label=label2)

    ax.set_xlim(1979 - 10, 2023 + 10)  # leave some space on either side
    ax.set_xlabel(i_to_month(i + 1))
    ax.set_xticks([])
    ax.set_frame_on(False)
    ax.grid(True)
    ax.tick_params(left=False, bottom=False)
    if i == 0:
        ax.set_ylabel("CO$_2$ concentration (ppm)")
plt.suptitle("CO$_2$: deviation from quadratic trend")
fig.legend()
plt.show()